# Tensorboard for Visualization

In [20]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path

# Reset the name counter
def cls():
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)

In [2]:
housing = datasets.fetch_california_housing()
x_train, x_test, y_train, y_test   = train_test_split(housing.data, housing.target, random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, random_state=42)
print(x_train.shape, x_valid.shape, x_test.shape)

(11610, 8) (3870, 8) (5160, 8)


In [3]:
cls()
norm_layer = keras.layers.Normalization(input_shape=x_train.shape[1:])
model = keras.Sequential([
    norm_layer, keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3), metrics=["RootMeanSquaredError"])
norm_layer.adapt(x_train)

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 8)                 17        
_________________________________________________________________
dense (Dense)                (None, 30)                270       
_________________________________________________________________
dense_1 (Dense)              (None, 30)                930       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 1,248
Trainable params: 1,231
Non-trainable params: 17
_________________________________________________________________


In [7]:
tensorboard_path = f'tensorboard/{datetime.now().strftime("_%Y_%m_%d_%H_%M_%S")}'
cb_tensorboard = keras.callbacks.TensorBoard(tensorboard_path)
history = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=10,callbacks=[cb_tensorboard])

Epoch 1/10
363/363 [==============================] - 1s 2ms/step - loss: 1.8866 - root_mean_squared_error: 1.3736 - val_loss: 0.7126 - val_root_mean_squared_error: 0.8442
Epoch 2/10
363/363 [==============================] - 1s 1ms/step - loss: 0.6577 - root_mean_squared_error: 0.8110 - val_loss: 0.6880 - val_root_mean_squared_error: 0.8295
Epoch 3/10
363/363 [==============================] - 1s 1ms/step - loss: 0.5934 - root_mean_squared_error: 0.7703 - val_loss: 0.5803 - val_root_mean_squared_error: 0.7618
Epoch 4/10
363/363 [==============================] - 1s 1ms/step - loss: 0.5557 - root_mean_squared_error: 0.7455 - val_loss: 0.5166 - val_root_mean_squared_error: 0.7188
Epoch 5/10
363/363 [==============================] - 1s 1ms/step - loss: 0.5272 - root_mean_squared_error: 0.7261 - val_loss: 0.4895 - val_root_mean_squared_error: 0.6997
Epoch 6/10
363/363 [==============================] - 1s 1ms/step - loss: 0.5033 - root_mean_squared_error: 0.7094 - val_loss: 0.4951 - val_

In [8]:
for idx, path in enumerate(sorted(Path("tensorboard").glob("**/*"))):
    if idx>20:
        break
    print("  " * (len(path.parts) - 1) + path.parts[-1])

  _2022_06_02_20_11_27
    train
      events.out.tfevents.1654180887.RPL-RAJ-105.22428.0.v2
      events.out.tfevents.1654180887.RPL-RAJ-105.profile-empty
      plugins
        profile
          2022_06_02_14_41_27
            RPL-RAJ-105.input_pipeline.pb
            RPL-RAJ-105.kernel_stats.pb
            RPL-RAJ-105.memory_profile.json.gz
            RPL-RAJ-105.overview_page.pb
            RPL-RAJ-105.tensorflow_stats.pb
            RPL-RAJ-105.trace.json.gz
            RPL-RAJ-105.xplane.pb
    validation
      events.out.tfevents.1654180888.RPL-RAJ-105.22428.1.v2
  _2022_06_02_20_12_19
    train
      events.out.tfevents.1654180940.RPL-RAJ-105.22428.2.v2
      events.out.tfevents.1654180940.RPL-RAJ-105.profile-empty
      plugins


In [9]:
%load_ext tensorboard
%tensorboard --logdir=./tensorboard 

http://localhost:6006/  <-- Go to this

In [11]:
# Till now, we just wrote some scaler values over there. You can do more things as well.

# To visualize histogram, to listen audio, image, text
import numpy as np
tensorboard_path = f'tensorboard/{datetime.now().strftime("_%Y_%m_%d_%H_%M_%S")}'
writer = tf.summary.create_file_writer(str(tensorboard_path))
with writer.as_default():
    for step in range(1, 1000 + 1):
        tf.summary.scalar("my_scalar", np.sin(step / 10), step=step)
        
        data = (np.random.randn(100) + 2) * step / 100  # gets larger
        tf.summary.histogram("my_hist", data, buckets=50, step=step)
        
        images = np.random.rand(2, 32, 32, 3) * step / 1000  # gets brighter
        tf.summary.image("my_images", images, step=step)
        
        texts = ["The step is " + str(step), "Its square is " + str(step ** 2)]
        tf.summary.text("my_text", texts, step=step)
        
        sine_wave = tf.math.sin(tf.range(12000) / 48000 * 2 * np.pi * step)
        audio = tf.reshape(tf.cast(sine_wave, tf.float32), [1, -1, 1])
        tf.summary.audio("my_audio", audio, sample_rate=48000, step=step)

You can share your TensorBoard logs with the world by uploading them to https://tensorboard.dev/. For this, you can run the tensorboard dev upload command, with the --logdir and --one_shot options, and optionally the --name and --description options. The first time, it will ask you to accept Google's Terms of Service, and to authenticate. This requires user input. Colab supports user input from shell commands, but the main other Jupyter environments do not, so for them we use a hackish workaround (alternatively, you could run the command in a terminal window, after you make sure to activate this project's conda environment and move to this notebook's directory).

In [16]:
from tensorboard.main import run_main
import sys

argv = "tensorboard dev upload --logdir ./tensorboard --one_shot".split()
argv += ["--name", "Quick test", "--description", "This is a test"]
try:
    original_sys_argv_and_sys_exit = sys.argv, sys.exit
    sys.argv, sys.exit = argv, lambda status: None
    run_main()
finally:
    sys.argv, sys.exit = original_sys_argv_and_sys_exit

In [ ]:
!tensorboard dev list

In [ ]:
# To list down all active states
from tensorboard import notebook
notebook.list()

# Fine Tuning the Neural Network

The Keras Tuner has four tuners available - RandomSearch, Hyperband, BayesianOptimization, and Sklearn.

### 01. Using Random Search

In [64]:
import keras_tuner as kt

def build_model(hp):
    n_hidden      = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons     = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,sampling="log")
    optimizer     = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    model = keras.Sequential()
    model.add(keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [25]:
random_search_tuner = kt.RandomSearch(build_model, objective="val_accuracy", max_trials=3, overwrite=True,
    directory="california_house", project_name="my_rnd_search", seed=42)
random_search_tuner.search(x_train, y_train, epochs=4, validation_data=(x_valid, y_valid))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.004392764996737242

Best val_accuracy So Far: 0.004651162773370743
Total elapsed time: 00h 00m 12s
INFO:tensorflow:Oracle triggered exit


I0603 08:38:13.061116 16832 <ipython-input-25-6130168ecc14>:3] Oracle triggered exit


In [26]:
top3_models = random_search_tuner.get_best_models(num_models=3)
best_model = top3_models[0]
best_model

In [30]:
top3_params = random_search_tuner.get_best_hyperparameters(num_trials=3)
top3_params[0].values  # best hyperparameter values

{'n_hidden': 5,
 'n_neurons': 25,
 'learning_rate': 0.0006562536901904111,
 'optimizer': 'adam'}

In [38]:
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial summary
Hyperparameters:
n_hidden: 5
n_neurons: 25
learning_rate: 0.0006562536901904111
optimizer: adam
Score: 0.004651162773370743


In [39]:
best_trial.metrics.get_last_value("val_accuracy")

0.004651162773370743

In [40]:
best_model.fit(x_train, y_train, epochs=10)
test_loss, test_accuracy = best_model.evaluate(x_test, y_test)
test_loss, test_accuracy

Epoch 1/10
363/363 [==============================] - 1s 2ms/step - loss: 1.5108 - accuracy: 0.0028
Epoch 2/10
363/363 [==============================] - 1s 1ms/step - loss: 1.4100 - accuracy: 0.0029
Epoch 3/10
363/363 [==============================] - 1s 1ms/step - loss: 1.3618 - accuracy: 0.0028
Epoch 4/10
363/363 [==============================] - 1s 2ms/step - loss: 1.3570 - accuracy: 0.0019
Epoch 5/10
363/363 [==============================] - 1s 1ms/step - loss: 1.2990 - accuracy: 0.0021
Epoch 6/10
363/363 [==============================] - 1s 2ms/step - loss: 1.2474 - accuracy: 0.0023
Epoch 7/10
363/363 [==============================] - 1s 2ms/step - loss: 1.2650 - accuracy: 0.0023
Epoch 8/10
363/363 [==============================] - 1s 2ms/step - loss: 1.2166 - accuracy: 0.0021
Epoch 9/10
363/363 [==============================] - 1s 2ms/step - loss: 1.1978 - accuracy: 0.0017
Epoch 10/10
162/162 [==============================] - 0s 1ms/step - loss: 1.1402 - accuracy: 0.0016

(1.1401960849761963, 0.001550387591123581)

### 02. Using Keras Wrapper of scikit learn

In [50]:
def build_model(n_hidden=1, n_neurons=10, learning_rate=1e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [51]:
# This regressor will use default parameters
keras_regressor = keras.wrappers.scikit_learn.KerasRegressor(build_model)
keras_regressor.fit(x_train, y_train, epochs=4, validation_data=(x_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=2)])

Epoch 1/4
363/363 [==============================] - 1s 2ms/step - loss: 1909799413469987995648.0000 - val_loss: 541821669262491648.0000
Epoch 2/4
363/363 [==============================] - 1s 2ms/step - loss: 286294118995329024.0000 - val_loss: 126656876162056192.0000
Epoch 3/4
363/363 [==============================] - 0s 1ms/step - loss: 66924463874113536.0000 - val_loss: 29607445101805568.0000
Epoch 4/4
363/363 [==============================] - 0s 1ms/step - loss: 15644358246137856.0000 - val_loss: 6921086394433536.0000


In [52]:
mse_test = keras_regressor.score(x_test, y_test)
mse_test

162/162 [==============================] - 0s 877us/step - loss: 6921085857562624.0000


-6921085857562624.0

In [53]:
# To hypertune the model
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

In [58]:
params = {"n_hidden":[1,2,3,4], "n_neurons":[5,10,20], 'learning_rate':reciprocal(3e-4, 3e-2)}
rn_search = RandomizedSearchCV(keras_regressor, param_distributions=params, n_iter=5, cv=3)
rn_search.fit(x_train, y_train, epochs=5, validation_data=(x_valid, y_valid))

Epoch 1/5
242/242 [==============================] - 1s 2ms/step - loss: nan - val_loss: nan
Epoch 2/5
242/242 [==============================] - 0s 1ms/step - loss: nan - val_loss: nan
Epoch 3/5
242/242 [==============================] - 0s 957us/step - loss: nan - val_loss: nan
Epoch 4/5
242/242 [==============================] - 0s 1ms/step - loss: nan - val_loss: nan
Epoch 5/5
121/121 [==============================] - 0s 799us/step - loss: nan
Epoch 1/5
242/242 [==============================] - 1s 2ms/step - loss: nan - val_loss: nan
Epoch 2/5
242/242 [==============================] - 0s 1ms/step - loss: nan - val_loss: nan
Epoch 3/5
242/242 [==============================] - 0s 943us/step - loss: nan - val_loss: nan
Epoch 4/5
242/242 [==============================] - 0s 778us/step - loss: nan - val_loss: nan
Epoch 5/5
121/121 [==============================] - 0s 780us/step - loss: nan
Epoch 1/5
242/242 [==============================] - 1s 2ms/step - loss: nan - val_loss: nan

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [            nan             nan -8.38654163e+18             nan
             nan]
  warnings.warn(


363/363 [==============================] - 1s 1ms/step - loss: 780.3945 - val_loss: 2.5867
Epoch 2/5
363/363 [==============================] - 0s 537us/step - loss: 1.9242 - val_loss: 1.4510
Epoch 3/5
363/363 [==============================] - 0s 662us/step - loss: 1.4133 - val_loss: 1.3253
Epoch 4/5
363/363 [==============================] - 0s 701us/step - loss: 1.3489 - val_loss: 1.3149
Epoch 5/5
363/363 [==============================] - 0s 723us/step - loss: 1.3407 - val_loss: 1.3155


RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x00000204993D7880>,
                   n_iter=5,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002049B637C40>,
                                        'n_hidden': [1, 2, 3, 4],
                                        'n_neurons': [5, 10, 20]})

In [60]:
rn_search.best_params_

{'learning_rate': 0.0014234029388933082, 'n_hidden': 1, 'n_neurons': 5}

In [61]:
best_model = rn_search.best_estimator_.model
best_model

### 03. Using HyperBand & Bayesian 

In [67]:
def build_model(hp):
    n_hidden      = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons     = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,sampling="log")
    optimizer     = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    model = keras.Sequential()
    model.add(keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [68]:
class MyClassificationHyperModel(kt.HyperModel):
    def build(self, hp):
        return build_model(hp)

    def fit(self, hp, model, X, y, **kwargs):
        if hp.Boolean("normalize"):
            norm_layer = keras.layers.Normalization()
            X = norm_layer(X)
        return model.fit(X, y, **kwargs)

In [69]:
hyperband_tuner = kt.Hyperband(
    MyClassificationHyperModel(), objective="val_accuracy", seed=42,
    max_epochs=4, factor=3, hyperband_iterations=2,
    overwrite=True, directory="california_house", project_name="hyperband")

In [75]:
root_logdir = Path(hyperband_tuner.project_dir) / "tensorboard"
tensorboard_cb = tf.keras.callbacks.TensorBoard(root_logdir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=2)
hyperband_tuner.search(x_train, y_train, epochs=4, validation_data=(x_valid, y_valid), callbacks=[early_stopping_cb, tensorboard_cb])

Trial 20 Complete [00h 00m 03s]
val_accuracy: 0.004651162773370743

Best val_accuracy So Far: 0.004651162773370743
Total elapsed time: 00h 00m 43s
INFO:tensorflow:Oracle triggered exit


I0603 09:26:05.667347 16832 <ipython-input-75-40280a37c112>:4] Oracle triggered exit


In [76]:
# Instead of HyperBand you can use BayesianOptimization as well
bayesian_opt_tuner = kt.BayesianOptimization(
    MyClassificationHyperModel(), objective="val_accuracy", seed=42,
    max_trials=10, alpha=1e-4, beta=2.6,
    overwrite=True, directory="california_house", project_name="bayesian_opt")
bayesian_opt_tuner.search(x_train, y_train, epochs=4,
                          validation_data=(x_valid, y_valid),
                          callbacks=[early_stopping_cb])

Trial 10 Complete [00h 00m 02s]
val_accuracy: 0.0036175709683448076

Best val_accuracy So Far: 0.004392764996737242
Total elapsed time: 00h 00m 31s
INFO:tensorflow:Oracle triggered exit


I0603 09:28:09.447609 16832 <ipython-input-76-ad5ee306d8ba>:6] Oracle triggered exit


In [77]:
%tensorboard --logdir {root_logdir}